In [2]:
import pandas as pd
# import numpy as np
import requests as req
import sys
import time

# pip install holidays if you don't have the package already
# import holidays

from datetime import datetime
from datetime import timedelta
# change the following filepath, file, and key name to your own
sys.path.insert(0, '../../API_Keys/')
from vantage_api import alpha_vantage_key

import pickle

In [3]:
#below, be sure to change the name of 'apikey' to your key's name
def pull_all_stocks(df):
    stock_dict = {}
    for ticker in list(df['ticker'].unique()):
        json = req.get(url = "https://www.alphavantage.co/query",
                  params = {
                      'function': 'TIME_SERIES_DAILY',
                      'outputsize': 'full',
                      'symbol': ticker,
                      'apikey':alpha_vantage_key #change the name of the key to your key name
                  }).json()
        ticker_df = pd.DataFrame(json['Time Series (Daily)']).T
        ticker_df.rename(columns = {'1. open': 'open',
                                 '2. high': 'high',
                                 '3. low': 'low',
                                 '4. close': 'close',
                                 '5. volume': 'volume'}, inplace = True)
        if ticker_df.shape[0] > 2500:
            ticker_df = ticker_df.iloc[range(2500)] #Data beyond 2012 is unnecessary, this helps reduce file size
        stock_dict[f'{ticker}'] = ticker_df
        time.sleep(2) #Might need to be changed for long list of tickers
    return stock_dict


In [7]:
reps = pd.read_csv('data/cleaned_complete_congress_data.csv')
reps.head()

,transaction_date,ticker,asset_description,type,amount,name,disclosure_date,chamber,first_name,last_name,party,birthday,gender,represents
0,2022-06-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2022-06-13,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY
1,2022-05-31,X,"United States Steel Corporation <div class=""te...",sale_partial,50000,Thomas H Tuberville,2022-06-13,senate,Thomas,Tuberville,Republican,1954-09-18,M,AL
2,2022-05-31,X,"United States Steel Corporation <div class=""te...",sale_full,50000,Thomas H Tuberville,2022-06-13,senate,Thomas,Tuberville,Republican,1954-09-18,M,AL
3,2022-05-31,X,United States Steel Corporation,purchase,250000,Thomas H Tuberville,2022-06-13,senate,Thomas,Tuberville,Republican,1954-09-18,M,AL
4,2022-05-20,PYPL,"PayPal Holdings, Inc. -",purchase,100000,Thomas H Tuberville,2022-06-13,senate,Thomas,Tuberville,Republican,1954-09-18,M,AL


In [44]:
mcconnell = reps[reps['last_name'] == 'McConnell']
mcconnell.head()

,transaction_date,ticker,asset_description,type,amount,name,disclosure_date,chamber,first_name,last_name,party,birthday,gender,represents
0,2022-06-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2022-06-13,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY
111,2022-03-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2022-03-14,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY
355,2021-12-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2021-12-16,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY
455,2021-09-03,WFC,WFC - Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2021-09-15,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY
669,2021-06-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2021-06-07,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY


In [9]:
mcconnell.shape

(23, 14)

In [10]:
stocks = pull_all_stocks(mcconnell)

Because I'm traveling and won't always have internet access, I wanted to save the stocks dictionary on my computer. [This site](https://www.geeksforgeeks.org/save-a-dictionary-to-a-file/) walked me through how to do that. I opted for pickle since we just learned that yesterday.

In [14]:
try:
    stocks_file = open('stocks_file', 'wb')
    pickle.dump(stocks, stocks_file)
    stocks_file.close()
  
except:
    print("Something went wrong")

In [12]:
#testing this out

with open('stocks_file', 'rb') as f:
    stocks = pickle.load(f)

In [ ]:
# sanity check. looks good.
stocks2

{'WFC':                open     high      low    close    volume
 2022-07-25  43.5300  43.9100  43.2100  43.5000  20297871
 2022-07-22  43.1700  43.5300  42.6900  43.1700  15965371
 2022-07-21  42.4800  43.3700  42.2800  43.2800  17808953
 2022-07-20  42.5400  43.0250  42.3550  42.7200  19111193
 2022-07-19  41.8800  43.0700  41.7450  42.9000  25724214
 ...             ...      ...      ...      ...       ...
 2012-08-22  34.3300  34.4600  34.1100  34.1800  18862500
 2012-08-21  34.2400  34.7600  34.1000  34.3800  25280400
 2012-08-20  34.0300  34.2000  33.9800  34.0700  18147600
 2012-08-17  34.2500  34.3000  33.9600  34.0300  16927000
 2012-08-16  34.1200  34.2000  33.7800  34.1300  19596400
 
 [2500 rows x 5 columns],
 'VMC':                 open      high       low     close  volume
 2022-07-25  156.3200  156.8000  154.8600  156.4000  591024
 2022-07-22  157.1600  157.8200  154.1300  155.6500  480375
 2022-07-21  153.8600  156.6500  153.1450  156.4600  592335
 2022-07-20  151.5200 

In [13]:
stocks

{'WFC':                open     high      low    close    volume
 2022-07-25  43.5300  43.9100  43.2100  43.5000  20297871
 2022-07-22  43.1700  43.5300  42.6900  43.1700  15965371
 2022-07-21  42.4800  43.3700  42.2800  43.2800  17808953
 2022-07-20  42.5400  43.0250  42.3550  42.7200  19111193
 2022-07-19  41.8800  43.0700  41.7450  42.9000  25724214
 ...             ...      ...      ...      ...       ...
 2012-08-22  34.3300  34.4600  34.1100  34.1800  18862500
 2012-08-21  34.2400  34.7600  34.1000  34.3800  25280400
 2012-08-20  34.0300  34.2000  33.9800  34.0700  18147600
 2012-08-17  34.2500  34.3000  33.9600  34.0300  16927000
 2012-08-16  34.1200  34.2000  33.7800  34.1300  19596400
 
 [2500 rows x 5 columns],
 'VMC':                 open      high       low     close  volume
 2022-07-25  156.3200  156.8000  154.8600  156.4000  591024
 2022-07-22  157.1600  157.8200  154.1300  155.6500  480375
 2022-07-21  153.8600  156.6500  153.1450  156.4600  592335
 2022-07-20  151.5200 

In [15]:
mcconnell.dtypes

transaction_date     object
ticker               object
asset_description    object
type                 object
amount                int64
name                 object
disclosure_date      object
chamber              object
first_name           object
last_name            object
party                object
birthday             object
gender               object
represents           object
dtype: object

# Code to Pull Stock Prices
[Stackoverflow](https://stackoverflow.com/a/61426301) on mean in multipe columns. helped to get this written.[Stackoverflow](https://stackoverflow.com/a/61426301) on mean in multipe columns. helped to get this written.

## Transaction Date = Day Market Was Closed
Congresspeople sometimes entered transaction dates for which the market was closed. We presumed that means they entered the trade with their broker (or otherwise) on that date, and that it would be executed the next trading day. That's accounted for within a couple of try/except pairs.

[This stackoverflow](https://stackoverflow.com/a/61964191) helped with setting up the try/except. [This medium article](https://medium.com/analytics-vidhya/python-how-to-automate-bank-holidays-767de0d42d44) showed how us the holidays module. [This article](https://www.geeksforgeeks.org/python-holidays-library/) helped understand how to use it. [This](https://pynative.com/python-get-the-day-of-week/) showed how to get day of week to check if saturday or sunday so we can adjust day of transaction accordingly.

## Market Closed non-Holidays
2018-12-05 the market was closed for national day of mourning to mark [the passing of George Bush](https://heavy.com/news/2018/12/stock-market-closed-national-day-of-mourning/). Need to add it to the list of holidays.# Market Closed non-Holidays
2018-12-05 the market was closed for national day of mourning to mark [the passing of George Bush](https://heavy.com/news/2018/12/stock-market-closed-national-day-of-mourning/). Need to add it to the list of holidays.

We presume the holiday transaction date means that the orders were executed the next market day.

## Trades When Stocks Weren't Trading AKA Mitch McConnell Sells Something Not On the Exchange?
Some trades are registered as happening when stocks weren't trading. Specifically Ingersoll-Rand (IR) partial_sales by Mitch McConnell 2017-02-02 and 2016-06-09. 

The data on IR doesn't start until May 8, 2017. [This Motley Fool article](https://www.fool.com/investing/general/2016/01/04/is-ingersoll-rand-a-buy-for-2016.aspxhttps://www.fool.com/investing/general/2016/01/04/is-ingersoll-rand-a-buy-for-2016.aspx) makes it appear that Ingersoll-Rand was trading under the ticker TT (or perhaps that the part that wasn't spun into IR is trading as TT now).

Gardner Denver went public in May 2017. In May 2020, it combined with Ingersoll-Rand's HVAC business -- together they started trading under IR. See [this article](https://seekingalpha.com/article/4350361-new-ingersoll-rand-debuts-under-challenging-circumstanceshttps://seekingalpha.com/article/4350361-new-ingersoll-rand-debuts-under-challenging-circumstances).

This may be a noteworthy trade. The disclosure wasn't made until May 15, 2017 -- about the time of the appearance of IR on the stock exchange, and the time of Gardner Denver going public, but before they merged (at least, that it was announced).

Weird.

He sold it in June of 2019... we should look at this more closely. In the meantime, I'll estimate it as being from the first IR price.

[This site](https://docs.python.org/2/library/datetime.html#datetime.timedeltahttps://docs.python.org/2/library/datetime.html#datetime.timedelta) showed us timedelta

[This stackoverflow answer](https://stackoverflow.com/a/42380166) (finally) showed us how to use `.loc()` to create a new column without getting a copy warning. (in this case filled with a default value). It still seems to throw a warning, but maybe not as many of them. Though we seem to be doing what the warning tells us to do... YEAH, ACTUALLY THIS DOESN'T SEEM TO MAKE ANY DIFFERENCE TO THE WARNING. Weird.

### A note on price column names

We've included final_price for EDA purposes, though we don't use it in the below calculations. One can determine if it's associated with a sale or not from `'type'` column.

We've incuded start_price instead of purchase price because sometimes the start price is day 1 of the data set and sometimes it's the price on a day of purchse. One can determine which from the `'type'` column.

We've used the same nomenclature for the prices, 'start' and 'final'.

In [45]:
t0 = time.time()

#revising to get 'start_price' and 'final_price'... use the piece from below

#drop partial sales for this version because we can't account for how much
#was sold

mcconnell = mcconnell[mcconnell['type'] != 'sale_partial']

#establish columns for purchase date and final date,
#which is either the sale date or the last date we're keeping records for

mcconnell.loc[:, 'start_date'] = ''
mcconnell.loc[:, 'final_date'] = ''

#establish columns for purchase price and final price,
#which is either the sale price or the price on the last date we're keeping records for

mcconnell.loc[:, 'start_price'] = 0
mcconnell.loc[:, 'final_price'] = 0

#establish purchase-sale column and setting default to 0
mcconnell.loc[:, 'purchase-sale'] = 0

#establishing all the dates we cover in this period
#TROUBLESHOOT THIS
start_date = pd.to_datetime(reps["transaction_date"].min())
end_date = pd.to_datetime('2022-07-16') #the data was pulled on 2022-07-15. 2022-07-06 here ensures that date is in the final list
date_list = [start_date + timedelta(days = num) for num in range((end_date-start_date).days)]

date_list = [str(date)[:10] for date in date_list]

for i in range(len(date_list)):
    mcconnell.loc[:, date_list[i]] = 0

print('*'*100)
print('')
print('Finished filling in the dates with 0s')
print('')
print("Time to run", time.time()-t0) 

/var/folders/5_/498w081d1837cc82dmb4hwmc0000gn/T/ipykernel_7882/1175688921.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mcconnell.loc[:, date_list[i]] = 0
/var/folders/5_/498w081d1837cc82dmb4hwmc0000gn/T/ipykernel_7882/1175688921.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mcconnell.loc[:, date_list[i]] = 0
/var/folders/5_/498w081d1837cc82dmb4hwmc0000gn/T/ipykernel_7882/1175688921.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which ha

Finished filling in the dates with 0s


/var/folders/5_/498w081d1837cc82dmb4hwmc0000gn/T/ipykernel_7882/1175688921.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mcconnell.loc[:, date_list[i]] = 0
/var/folders/5_/498w081d1837cc82dmb4hwmc0000gn/T/ipykernel_7882/1175688921.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mcconnell.loc[:, date_list[i]] = 0
/var/folders/5_/498w081d1837cc82dmb4hwmc0000gn/T/ipykernel_7882/1175688921.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which ha

[This site](https://www.geeksforgeeks.org/python-iterating-through-a-range-of-dates/) showed us the date_range method.

In [43]:
t0 = time.time()

for i in mcconnell.index:
    print(i)
    #establish ticker
    ticker = mcconnell.loc[i, 'ticker']
    start_date = 0
    final_date = 0
    
    #this accounts for purchases
    if mcconnell.loc[i, 'type'] == 'purchase':
        
        start_date = mcconnell.loc[i, 'transaction_date']
        print(f'Index {i} is a purchase and the purchase date is {start_date}')
        
        for n in mcconnell.index:
            
            #determine if there's a full sale after the purchase:
            if (mcconnell.loc[n, 'ticker'] == ticker) and (mcconnell.loc[n, 'type'] == 'sale_full') and (pd.to_datetime(mcconnell.loc[n, 'transaction_date']) > pd.to_datetime(start_date)):
                print(f'Index {i} is a purchase that is later sold')
                #NOTE: THIS CODE DOESN'T YET ACCOUNT FOR A REP BUYING, SELLING, THEN DOING IT AGAIN LATER
                #WITH THE SAME TICKER
                
                #mark this row as a purchase --> sale:
                mcconnell.loc[i, 'purchase-sale'] = 1
                
                final_date = mcconnell.loc[n, 'transaction_date']
                
                mcconnell.drop(index = n, inplace = true)
                #include this to enable sanity check
                print(f"Dropped sale at index {n} because it's accounted for in purchase at index {i}")                

        #if the final_date hasn't been changed by the first by an if in the loop-->        
        #final_date is a misnomer here but is used to simplify the code
        if final_date == 0:
            print(f'Index {i} is a purchase that is never sold')
            final_date = '2022-07-15'
    
    #this accounts for sales that aren't associated with purchases
    else:
        
        if mcconnell.loc[i, 'purchase-sale'] == 0:
            print(f'Index {i} is a sale with no associated purchase in the data')
            #assume purchase date is the first date for which we're tracking
            #this is very rough, but will at least let us track price changes
            start_date = '2012-09-13'
            
            final_date = mcconnell.loc[i, 'transaction_date']
    
                            #at this point, we've established the dates from which we'll work to find prices,
                            #which may not be the same because some transaction dates are days the market is closed

                            #next, we get actual day of purchase, which is the day of transaction (current 'start_date'),
                            #or the first day of our records, or the first day of trading after either of those
    
    print(f'Index {i} starting start (or purchase) date is {start_date}')
    count = 0
    while start_date not in stocks[ticker].index:
        start_date = str(pd.to_datetime(start_date) + timedelta(days = 1))[:10]
        count += 1
        if pd.to_datetime(start_date) > pd.to_datetime('2022-07-15'):
            print(f'For {ticker}, index {i} purchase date reached {date} without finding a price.')
            break
            
    print(f'Index {i} ending start (or purchase) date is {start_date}')       
    
    #next, we get actual day of sale, which is the day of transaction (current 'final_date'),
    #or the first day of trading after that, or the last day of our records   
    
    print(f'Index {i} starting final (or sale) date is {final_date}')
    count = 0
    while final_date not in stocks[ticker].index:
        final_date = str(pd.to_datetime(final_date) + timedelta(days = 1))[:10]
        count += 1
        if pd.to_datetime(final_date) > pd.to_datetime('2022-07-15'):
            print(f'For {ticker}, index {i} sale date reached {date} without finding a price.')
            break
    print(f'Index {i} ending final (or sale) date is {final_date}')
    
    mcconnell.loc[i, 'start_date'] = start_date
    mcconnell.loc[i, 'final_date'] = final_date
    
    #now we get the start_price and the final_price
    
    mcconnell.loc[i, 'start_price'] = stocks[ticker].loc[start_date].drop('volume').astype(float).mean()
    mcconnell.loc[i, 'final_price'] = stocks[ticker].loc[final_date].drop('volume').astype(float).mean()

    #final step is to fill in the deltas from between these two dates
    for date in pd.date_range(pd.to_datetime(start_date)+timedelta(days=1), pd.to_datetime(final_date)):
        try:
            price = stocks[ticker].loc[str(date)[:10]].drop('volume').astype(float).mean()
            mcconnell.loc[i, str(date)[:10]] = (price-mcconnell.loc[i, 'start_price'])/mcconnell.loc[i, 'start_price'] #(new-old)/old
        except:
            pass
print('')
print("Time to run", time.time()-t0)  

0
Index 0 is a purchase and the purchase date is 2022-06-03
Index 0 is a purchase that is never sold
Index 0 starting start (or purchase) date is 2022-06-03
Index 0 ending start (or purchase) date is 2022-06-03
Index 0 starting final (or sale) date is 2022-07-15
Index 0 ending final (or sale) date is 2022-07-15
111
Index 111 is a purchase and the purchase date is 2022-03-03
Index 111 is a purchase that is never sold
Index 111 starting start (or purchase) date is 2022-03-03
Index 111 ending start (or purchase) date is 2022-03-03
Index 111 starting final (or sale) date is 2022-07-15
Index 111 ending final (or sale) date is 2022-07-15
355
Index 355 is a purchase and the purchase date is 2021-12-03
Index 355 is a purchase that is never sold
Index 355 starting start (or purchase) date is 2021-12-03
Index 355 ending start (or purchase) date is 2021-12-03
Index 355 starting final (or sale) date is 2022-07-15
Index 355 ending final (or sale) date is 2022-07-15
455
Index 455 is a purchase and t

/var/folders/5_/498w081d1837cc82dmb4hwmc0000gn/T/ipykernel_7882/1995714237.py:80: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mcconnell.loc[i, 'start_date'] = start_date
/var/folders/5_/498w081d1837cc82dmb4hwmc0000gn/T/ipykernel_7882/1995714237.py:85: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mcconnell.loc[i, 'start_price'] = stocks[ticker].loc[start_date].drop('volume').astype(float).mean()



Index 669 is a purchase and the purchase date is 2021-06-03
Index 669 is a purchase that is never sold
Index 669 starting start (or purchase) date is 2021-06-03
Index 669 ending start (or purchase) date is 2021-06-03
Index 669 starting final (or sale) date is 2022-07-15
Index 669 ending final (or sale) date is 2022-07-15
709
Index 709 is a purchase and the purchase date is 2021-03-03
Index 709 is a purchase that is never sold
Index 709 starting start (or purchase) date is 2021-03-03
Index 709 ending start (or purchase) date is 2021-03-03
Index 709 starting final (or sale) date is 2022-07-15
Index 709 ending final (or sale) date is 2022-07-15
828
Index 828 is a purchase and the purchase date is 2020-12-03
Index 828 is a purchase that is never sold
Index 828 starting start (or purchase) date is 2020-12-03
Index 828 ending start (or purchase) date is 2020-12-03
Index 828 starting final (or sale) date is 2022-07-15
Index 828 ending final (or sale) date is 2022-07-15
966
Index 966 is a pur

In [111]:
pd.set_option('display.max_columns', 100)

In [39]:
mcconnell

,transaction_date,ticker,asset_description,type,amount,name,disclosure_date,chamber,first_name,last_name,...,2022-07-06,2022-07-07,2022-07-08,2022-07-09,2022-07-10,2022-07-11,2022-07-12,2022-07-13,2022-07-14,2022-07-15
0,2022-06-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2022-06-13,senate,Mitchell,McConnell,...,-0.119856,-0.106047,-0.103318,0,0,-0.116292,-0.119327,-0.131856,-0.148003,-0.100339
111,2022-03-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2022-03-14,senate,Mitchell,McConnell,...,-0.236733,-0.224757,-0.222391,0,0,-0.233642,-0.236274,-0.247139,-0.261142,-0.219808
355,2021-12-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2021-12-16,senate,Mitchell,McConnell,...,-0.183718,-0.170910,-0.168379,0,0,-0.180412,-0.183227,-0.194847,-0.209822,-0.165617
455,2021-09-03,WFC,WFC - Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2021-09-15,senate,Mitchell,McConnell,...,-0.106877,-0.092864,-0.090095,0,0,-0.103260,-0.106340,-0.119054,-0.135439,-0.087072
669,2021-06-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2021-06-07,senate,Mitchell,McConnell,...,-0.152467,-0.139169,-0.136542,0,0,-0.149035,-0.151957,-0.164022,-0.179571,-0.133673
709,2021-03-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2021-03-16,senate,Mitchell,McConnell,...,0.046441,0.062860,0.066104,0,0,0.050679,0.047071,0.032174,0.012976,0.069646
828,2020-12-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2020-12-07,senate,Mitchell,McConnell,...,0.358179,0.379490,0.383700,0,0,0.363679,0.358996,0.339662,0.314745,0.388297
966,2020-09-02,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2020-09-11,senate,Mitchell,McConnell,...,0.626684,0.652207,0.657250,0,0,0.633272,0.627663,0.604507,0.574663,0.662756
1996,2020-03-04,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2020-03-11,senate,Mitchell,McConnell,...,-0.035013,-0.019872,-0.016880,0,0,-0.031105,-0.034432,-0.048169,-0.065873,-0.013614
2196,2019-12-04,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2019-12-12,senate,Mitchell,McConnell,...,-0.253918,-0.242212,-0.239899,0,0,-0.250897,-0.253469,-0.264090,-0.277778,-0.237374


In [60]:
mcconnell.columns

Index(['transaction_date', 'ticker', 'asset_description', 'type', 'amount',
       'name', 'disclosure_date', 'chamber', 'first_name', 'last_name',
       ...
       '2022-07-06', '2022-07-07', '2022-07-08', '2022-07-09', '2022-07-10',
       '2022-07-11', '2022-07-12', '2022-07-13', '2022-07-14', '2022-07-15'],
      dtype='object', length=3612)

In [61]:
#maybe explore further -- should we start our count at a later date?
#also for EDA, could be interesting to look at these outlier dates. What was happening?
#for example, 2020-03-18 was right at the start of lockdown. Did it fall on? after?

reps['transaction_date'].value_counts()

2020-03-18    305
2019-06-24    208
2020-04-02    186
2021-02-16    184
2020-04-14    162
             ... 
2017-08-25      1
2018-03-22      1
2021-06-26      1
2015-12-30      1
2018-04-06      1
Name: transaction_date, Length: 1896, dtype: int64

In [47]:
#maybe explore further -- should we start our count at a later date?
#also for EDA, could be interesting to look at these outlier dates. What was happening?
#for example, 2020-03-18 was right at the start of lockdown. Did it fall on? after?

reps['transaction_date'].value_counts()

2020-03-18    305
2019-06-24    208
2020-04-02    186
2021-02-16    184
2020-04-14    162
             ... 
2017-08-25      1
2018-03-22      1
2021-06-26      1
2015-12-30      1
2018-04-06      1
Name: transaction_date, Length: 1896, dtype: int64

I'm eliminating the NaNs as I don't see any way to fill that data in right now.

In [40]:
mcconnell.isnull().sum()[:20]

transaction_date     0
ticker               0
asset_description    0
type                 0
amount               0
name                 0
disclosure_date      0
chamber              0
first_name           0
last_name            0
party                0
birthday             0
gender               0
represents           0
purchase_date        0
final_date           0
purchase_price       0
final_price          0
purchase-sale        0
2012-09-13           0
dtype: int64

In [41]:
mcconnell['type'].value_counts()

purchase     19
sale_full     2
Name: type, dtype: int64

In [55]:
 #a draft from above maybe useful elsewhere?
    #HOWEVER, IN ANOTHER ANALYSIS WE MAY WANT TO KEEP THESE TO LOOK AT PATTERNS OF
                                        #PRICES DROPPING OR NOT AFTER SALES
  #              for date in pd.date_range(sale_date + timedelta(days = 1, datetime('2022-07-15'))):
   #                 mcconnell.loc[i, [str(date)[:10]] = 0

In [56]:
mcconnell.columns[:20]

Index(['transaction_date', 'ticker', 'asset_description', 'type', 'amount',
       'name', 'disclosure_date', 'chamber', 'first_name', 'last_name',
       'party', 'birthday', 'gender', 'represents', 'trans_trading_day',
       'trading_day', 'price', 'purchase-sale', 'price_missing'],
      dtype='object')

In [192]:
stocks['IR']#.loc['2017-02-02']#.loc[mcconnell.loc[0, 'transaction_date']]

,open,high,low,close,volume
2022-07-22,45.0000,45.1900,44.1800,44.5400,2200156
2022-07-21,44.0700,44.9900,43.7300,44.9600,1633376
2022-07-20,43.2900,44.2400,43.1400,44.0700,2715744
2022-07-19,41.8300,43.8100,41.8300,43.2700,4499483
2022-07-18,41.5000,42.1600,40.9200,41.2200,3052687
...,...,...,...,...,...
2017-05-18,22.1900,23.6300,21.9800,22.5000,4075173
2017-05-17,22.1500,22.5100,21.6700,22.1000,1583699
2017-05-16,22.1900,22.4100,22.0500,22.1000,1289339
2017-05-15,21.0100,22.3000,21.0100,22.0200,3487278


In [180]:
stocks[mcconnell.loc[0, 'ticker']].loc['2017-02-02']#.loc[mcconnell.loc[0, 'transaction_date']]

open       55.5650
high       55.9600
low        55.4000
close      55.7500
volume    13044449
Name: 2017-02-02, dtype: object

In [ ]:
stocks[mcconnell.loc[i, 'ticker']].loc[mcconnell.loc[i, date_plus_2]]

In [174]:
stocks[mcconnell.loc[0, 'ticker']].loc[mcconnell.loc[0, 'transaction_date']].drop('volume').astype(float)

open     44.8300
high     45.3299
low      44.6100
close    44.8200
Name: 2022-06-03, dtype: float64

This pulls the average of open, high, low, and close. Using this because we have no idea when in the day the purchase was made.

In [176]:
stocks[mcconnell.loc[0, 'ticker']].loc[mcconnell.loc[0, 'transaction_date']].drop('volume').astype(float).mean()

44.897475